<a href="https://colab.research.google.com/github/mirrormouse/machine_learning/blob/main/english_words.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

アルファベットと数字の変換準備（母音->子音に並べた。後は割とランダム）

In [306]:
import random
char="aiueomnhygrlbcdfkpstvwzxjq"
dic={}
for i in range(len(char)):
  dic[char[i]]=i
print(dic['c'])

13


ドライブをインポート

In [307]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


でたらめな５文字からなる「英単語もどき」を生成。これを数値に変換
例）["aiueq",...]->[[0,1,2,3,25],...]

In [308]:
def generate(data_size=1814,length=5):
  random_list=[]
  random_words=[]
  for i in range(data_size):
    word=''
    num=[]
    for j in range(length):
      a=random.randrange(26)
      c=char[a]
      word+=c
      num.append(a)
    random_list.append(num)
    random_words.append(word)
  return random_list,random_words

lists,word=generate(data_size=1814)


英文データの読み込み

In [309]:
base='drive/MyDrive/ML/train/'
f = open(base+'train.txt', 'r')
rawdata = f.read()
data=rawdata.split()
print(data[:5])
f.close()

['The', 'two', 'submarines', 'of', 'the']


５文字の長さの英単語だけを大量に抽出し、「存在する英単語」としてまとめる。これも数値に変換。

In [310]:
import re
def correct_generate(data_size=10000,length=5):
  alpha_data=[]
  for word in data:
    word=word.lower()
    res = re.sub(r'[^a-zA-Z]', '', word)
    if len(res)==length:
      alpha_data.append(res)

  beta_data=[]
  for word in alpha_data:
    num=[]
    for id in range(len(word)):
      c=word[id]
      num.append(dic[c])
    beta_data.append(num)
  size=len(alpha_data)
  return alpha_data, beta_data, size

alpha,beta,_=correct_generate()
print(beta[:5])

[[13, 11, 0, 18, 18], [12, 2, 1, 11, 19], [0, 11, 4, 6, 9], [4, 19, 7, 3, 10], [19, 7, 3, 1, 10]]


訓練データの生成（上で作った２つを繋げるだけ）

In [311]:
x_train_list=lists+beta
y_train_list=[]
for i in range(len(lists)):
  y_train_list.append(0)
for i in range(len(beta)):
  y_train_list.append(1)

訓練データのリストをnumpyに変換。正解ラベルを生成

In [312]:
import numpy as np
x_train=np.array(x_train_list)
y_train=np.array(y_train_list)
print(x_train[:5],y_train[:5])

[[13 10  3 16 15]
 [14 24 11  9 12]
 [15 13 22  3 12]
 [ 3  2  8 10 20]
 [ 7 22  5  2 11]] [0 0 0 0 0]


ディープラーニングモデルを作る

In [313]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
model_1 = tf.keras.Sequential(name='my_model')
model_1.add(tf.keras.layers.Flatten(input_shape=(5,), name='flatten_layer_1'))
model_1.add(tf.keras.layers.Dense(128, activation='relu'))
model_1.add(tf.keras.layers.Dropout(0.2))
model_1.add(tf.keras.layers.Dense(128, activation='relu'))
model_1.add(tf.keras.layers.Dropout(0.2))
model_1.add(tf.keras.layers.Dense(128, activation='relu'))
model_1.add(tf.keras.layers.Dropout(0.2))
model_1.add(tf.keras.layers.Dense(128, activation='relu'))
model_1.add(tf.keras.layers.Dropout(0.2))
model_1.add(tf.keras.layers.Dense(128, activation='relu'))
model_1.add(tf.keras.layers.Dropout(0.2))
model_1.add(tf.keras.layers.Dense(128, activation='relu'))
model_1.add(tf.keras.layers.Dropout(0.2))
model_1.add(tf.keras.layers.Dense(2, activation='softmax'))

model_1.summary()

Model: "my_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_layer_1 (Flatten)    (None, 5)                 0         
_________________________________________________________________
dense_87 (Dense)             (None, 128)               768       
_________________________________________________________________
dropout_70 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_88 (Dense)             (None, 128)               16512     
_________________________________________________________________
dropout_71 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_89 (Dense)             (None, 128)               16512     
_________________________________________________________________
dropout_72 (Dropout)         (None, 128)               0  

モデルのコンパイル

In [321]:
model_1.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

学習！

In [322]:
history = model_1.fit(x_train, y_train, batch_size=1000, epochs=100,
                    validation_split=0.1)

Epoch 1/100
4/4 [==============================] - 1s 66ms/step - loss: 0.3477 - accuracy: 0.8487 - val_loss: 0.1711 - val_accuracy: 0.9559
Epoch 2/100
4/4 [==============================] - 0s 24ms/step - loss: 0.3426 - accuracy: 0.8496 - val_loss: 0.1839 - val_accuracy: 0.9504
Epoch 3/100
4/4 [==============================] - 0s 20ms/step - loss: 0.3357 - accuracy: 0.8530 - val_loss: 0.1801 - val_accuracy: 0.9504
Epoch 4/100
4/4 [==============================] - 0s 20ms/step - loss: 0.3525 - accuracy: 0.8484 - val_loss: 0.1785 - val_accuracy: 0.9559
Epoch 5/100
4/4 [==============================] - 0s 20ms/step - loss: 0.3444 - accuracy: 0.8499 - val_loss: 0.1662 - val_accuracy: 0.9587
Epoch 6/100
4/4 [==============================] - 0s 20ms/step - loss: 0.3361 - accuracy: 0.8570 - val_loss: 0.1733 - val_accuracy: 0.9559
Epoch 7/100
4/4 [==============================] - 0s 20ms/step - loss: 0.3480 - accuracy: 0.8478 - val_loss: 0.1794 - val_accuracy: 0.9559
Epoch 8/100
4/4 [===

In [329]:
def nothing_val(word):#元データ内にその単語が存在するか確認
  for w in alpha:
    if word==w:
      return False
  return True
nothing_val('joker')

True

単語とベクトルの変換用関数

In [324]:
def word2num(word):
  num=[]
  for id in range(len(word)):
    num.append(dic[word[id]])
  return num
def num2word(nums):
  word=''
  for i in nums:
    word+=char[i]
  return word
print(num2word([12, 3, 0, 18, 19]))
print(word2num("beast"))

beast
[12, 3, 0, 18, 19]


テスト　訓練データに含めていない"beast"や"blade"を正しく「存在する英単語」と判断していることが分かる

In [325]:
test_words=["beast","blade","joker","kdjus","adjsa"]
y_test=[1,1,1,0,0]
x_test=[]
for word in test_words:
  num=word2num(word)
  x_test.append(num)

test_loss, test_acc = model_1.evaluate(x_test, y_test, verbose=0)
print(test_loss)
print(test_acc)
predictions = model_1.predict(x_test)
print(predictions)

0.053424347192049026
1.0
[[0.09505937 0.90494066]
 [0.04268322 0.95731676]
 [0.10374782 0.89625216]
 [0.99711215 0.00288786]
 [0.98887306 0.01112699]]


大量のランダム文字列とペクトルを生成し、予測結果の「存在する単語」である確率が大きい順にソート

In [326]:
randomlist,randomword=generate(data_size=300000)
r_pred=model_1.predict(randomlist)
item=[]
for i in range(len(r_pred)):
  res=[i,r_pred[i][0]]
  item.append(res)
item.sort(key=lambda item:item[1])

予測結果が「正しい単語」に最も近い30単語を生成して表示。
なんとなく「英単語にありそう」な雰囲気の文字列があることがわかる（？）
("prase"は「緑石英」という意味だそうです。元データには入っていません。やったね！)

In [327]:
top=30
ans=[]
for i in range(top):
  id=item[i][0]
  ans.append([randomword[id],item[i][1]])
  print([randomword[id],item[i][1]])

['prase', 0.04011614]
['plapo', 0.040139616]
['dgace', 0.040400464]
['byace', 0.040451527]
['kgace', 0.040590085]
['kgace', 0.040590085]
['byafe', 0.040914893]
['cyalu', 0.04107978]
['zbavo', 0.041239113]
['fbape', 0.04125736]
['fraco', 0.0413994]
['rhabe', 0.041451566]
['sratm', 0.041512333]
['acaki', 0.041539103]
['byado', 0.041580204]
['dgabo', 0.04161989]
['prafm', 0.0417477]
['spavo', 0.04181343]
['flapm', 0.041819416]
['trade', 0.041900836]
['zcaxo', 0.042009134]
['lgade', 0.042012855]
['kcaso', 0.04204868]
['adavu', 0.042108234]
['jcawo', 0.04211275]
['tcate', 0.042118844]
['cgafm', 0.042122904]
['fpate', 0.04216026]
['tkavo', 0.042164136]
['zrato', 0.0421809]
